In [ ]:
!pip install flask flask-ngrok

In [ ]:
from google.colab import drive

# Load dataset
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import joblib
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanSquaredError
from pyngrok import ngrok

# Initialize Flask app
app = Flask(__name__)
CORS(app)  # Enable CORS for cross-origin requests

# Start ngrok tunnel
public_url = ngrok.connect(5000)
print(f"Public URL: {public_url}")

# Load models and scaler
iso_forest = joblib.load("/content/drive/MyDrive/anomaly_isolation_forest.pkl")
hybrid_autoencoder = load_model("/content/drive/MyDrive/hybrid_autoencoder.h5", custom_objects={"mse": MeanSquaredError()})
scaler = joblib.load("/content/drive/MyDrive/scaler.pkl")

print("Models and scaler loaded successfully.")

# Define manual thresholds
manual_thresholds = {
    'CO': (0, 50),
    'CO2': (0, 5000),
    'H2S': (0, 10),
    'NH3': (0, 25),
    'PR': (120, 200),
    'QT': (350, 450),
    'ST': (80, 120)
}

@app.route("/")
def home():
    return "Anomaly Detection API is running!"

@app.route("/predict", methods=["POST"])
def predict():
    try:
        # Get JSON data
        data = request.json["data"]

        # Ensure data is in DataFrame format for proper feature names
        feature_columns = ['CO', 'CO2', 'H2S', 'NH3', 'PR', 'QT', 'ST', 'HRV']
        input_df = pd.DataFrame([data], columns=feature_columns)

        # Scale the data
        input_data_scaled = scaler.transform(input_df)

        # Append iso_anomaly placeholder AFTER scaling
        input_data_final = np.column_stack((input_data_scaled, [[0]]))

        # Predict using Isolation Forest
        iso_anomaly = iso_forest.predict(input_data_final)
        iso_anomaly = 1 if iso_anomaly[0] == -1 else 0  # Convert to binary (0 or 1)

        # Prepare input for Hybrid Autoencoder
        hybrid_input = np.column_stack((input_data_scaled, [[iso_anomaly]]))
        hybrid_reconstructed = hybrid_autoencoder.predict(hybrid_input)
        hybrid_reconstruction_error = np.mean(np.abs(hybrid_input - hybrid_reconstructed), axis=1)

        # Define anomaly threshold for Hybrid Autoencoder
        threshold = 0.009
        hybrid_anomaly = 1 if hybrid_reconstruction_error[0] > threshold else 0

        # Apply manual thresholding
        manual_anomaly = any(
            not (low <= data[feature] <= high) for feature, (low, high) in manual_thresholds.items()
        )

        # Final anomaly decision
        final_anomaly = max(hybrid_anomaly, manual_anomaly)

        return jsonify({
            "IsolationForest_Anomaly": iso_anomaly,
            "Hybrid_Anomaly": hybrid_anomaly,
            "Manual_Threshold_Anomaly": int(manual_anomaly),
            "Final_Anomaly": final_anomaly
        })

    except Exception as e:
        return jsonify({"error": str(e)})

if __name__ == "__main__":
    app.run(port=5000)


Public URL: NgrokTunnel: "https://97b9-34-75-94-122.ngrok-free.app" -> "http://localhost:5000"
Models and scaler loaded successfully.
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step


INFO:werkzeug:127.0.0.1 - - [08/Apr/2025 09:21:41] "POST /predict HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step


...

INFO:werkzeug:127.0.0.1 - - [08/Apr/2025 09:24:21] "POST /predict HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step


INFO:werkzeug:127.0.0.1 - - [08/Apr/2025 09:24:28] "POST /predict HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step


INFO:werkzeug:127.0.0.1 - - [08/Apr/2025 09:24:35] "POST /predict HTTP/1.1" 200 -


In [ ]:
!pip install flask_cors


In [ ]:
!pip install pyngrok
from pyngrok import ngrok
!ngrok config add-authtoken 2t7eHroGCkLrnrgTlZkzdWEuVLP_2CWMekhkpnc894q1eXwLA

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
